In [10]:
from datasets import load_diabetes
from tools import *
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.express as px
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
np.set_printoptions(suppress=True)

In [11]:
df_original, df_train, df_test = load_diabetes()

In [12]:
train_X = df_train.drop(['target'], axis=1)
train_y = df_train['target']
test_X = df_test.drop(['target'], axis=1)
test_y = df_test['target']
original_X = df_original.drop(['target'], axis=1)
original_y = df_original['target']
features = train_X.columns

## Pocet komponent

In [13]:
pca = PCA(n_components=10)
pca = pca.fit(original_X)

In [14]:
# vidime ze nejvice variance popisuji prvni 3 komponenty, zvolime 3 pro lepsi vizualizaci
pca.explained_variance_ratio_

array([0.40242142, 0.14923182, 0.12059623, 0.09554764, 0.06621856,
       0.06027192, 0.05365605, 0.04336832, 0.00783199, 0.00085605])

## Regresion

In [18]:
pca = PCA(n_components=3)
pca = pca.fit(original_X)

In [19]:
transformed_original_X = pca.transform(original_X)
transformed_train_X = pca.transform(train_X)
transformed_test_X = pca.transform(test_X)

In [21]:
transformed_df = pd.DataFrame(transformed_original_X, columns=['x','y','z'])
transformed_df['target'] = original_y.values
px.scatter_3d(transformed_df, x='x', y='y', z='z',
              color='target', size='target')

In [22]:
cross_val_score(LinearRegression(), transformed_original_X, original_y).mean()

0.3443528362251431

In [23]:
model = LinearRegression()
model = model.fit(transformed_train_X, train_y)

In [24]:
model_performance(model, transformed_train_X, train_y, transformed_test_X, test_y)

Train score
0.36078746097619485
Test score
0.3927452602110665


## Overeni vyberu komponent

In [25]:
scores = []
components = []
for i in range(1, len(df_train.columns) - 1):
    pca = PCA(n_components=i)
    pca = pca.fit(original_X)
    transformed_train_X = pca.transform(train_X)
    transformed_test_X = pca.transform(test_X)
    components.append(i)
    scores.append(cross_val_score(LinearRegression(), transformed_train_X, train_y).mean())

pd.DataFrame(zip(components, scores), columns=['components', 'score'])

,components,score
0,1,0.279182
1,2,0.317261
2,3,0.324686
3,4,0.443801
4,5,0.444711
5,6,0.450161
6,7,0.456035
7,8,0.446627
8,9,0.450465
